In [2]:
pip install boto3

     |████████████████████████████████| 130 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 3.9 MB/s eta 0:00:011
     |████████████████████████████████| 7.2 MB 3.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
firehose=boto3.client('firehose',aws_access_key_id=AWS_KEY_ID, aws_access_secret_key= AW_SECRET, region_name= 'us_east_1')

NameError: name 'AWS_KEY_ID' is not defined

In [ ]:
response=firehose.list_delivery_streams()
print(response['DeliveryStreamNames']

In [ ]:
#Delete Old Stream Names
response= firehose.list_delivery_streams()
#Delete Them All
for stream_name in response['DeliveryStreamNames']
firehose.delete_delivery_stream(DeliveryStreamName=stream_name)
#another example

# Import boto3 and create boto3 Firehose client
import boto3
firehose = boto3.client('firehose', 
    aws_access_key_id=AWS_KEY_ID, aws_secret_access_key=AWS_SECRET, 
    region_name='us-east-1', endpoint_url=endpoints['FIREHOSE'])

# Get list of delivery streams
response = firehose.list_delivery_streams()

# Iterate over the response contents and delete every stream
for stream_name in response['DeliveryStreamNames']:
    firehose.delete_delivery_stream(DeliveryStreamName=stream_name)
    print(f"Deleted stream: {stream_name}")


# Print list of delivery streams
print(firehose.list_delivery_streams())

In [ ]:
#Creating Roles

s3.create_bucket(Bucket='sd-vehicle-data')

# List the buckets in S3
for bucket_info in s3.list_buckets()['Buckets']:
    
    # Get the bucket_name
    bucket_name = bucket_info['Name']
    
    # Generate bucket ARN.
    arn = "arn:aws:s3:::logs-bucket".format(bucket_name)
    
    # Print the ARN
    print(arn)

In [3]:
res=firehose.create_delivery_stream(DeliveryStreamName="gps-delivery-stream",
                                    DeliveryStreamType="DirectPut" # we are writing stream directly,
                                   ,S3DestinationConfiguration={"RoleARN": "...",
                                                                "BucketARN":"..."})
print(res['DeliveryStreamARN'])
#back with Delivery Stream ARN

ERROR: Invalid requirement: '_setup'
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# create_firehose.py: Create firehose stream. No need to edit.
import _setup
firehose, s3 = _setup.ex_vars

# Create s3 bucket
s3.create_bucket(Bucket='sd-vehicle-data')

# Create Firehose delivery stream
res = firehose.create_delivery_stream(
    DeliveryStreamName="gps-delivery-stream",
    DeliveryStreamType="DirectPut",
    # Specify configuration of the destination S3 bucket
    S3DestinationConfiguration = {
        "BucketARN": "arn:aws:s3:::sd-vehicle-data",
        "RoleARN": "arn:aws:iam::0000000:role/firehoseDeliveryRole"
    })
    
# Print the stream ARN
print("Created Firehose Stream ARN: {}".format(res['DeliveryStreamARN']))



In [ ]:
# OBD2_sensors.py: Write to Firehose stream. EDIT HERE.
import _setup, create_firehose
firehose, s3, records = _setup.ex_vars
for idx, row in records.iterrows(): 

    # Create a payload string that ends with a newline
    payload = ' '.join(str(value) for value in row) 
    payload = payload + "\n"
    print("Sending payload: {}".format(payload))

    # Send the payload string to Firehose stream
    res = firehose.put_record(
        DeliveryStreamName = 'gps-delivery-stream',
        Record = {'Data': payload})

    # Print the written RecordId
    print("Wrote to RecordId: {}".format(res['RecordId']))

In [ ]:
# List the objects that have been written to the S3 bucket
objects = s3.list_objects(Bucket='sd-vehicle-data')['Contents']

# Create list for collecting dataframes from read files.
dfs = []

# For every object, load it from S3.
for obj in objects:
    data_file = s3.get_object(Bucket='sd-vehicle-data', Key=obj['Key'])

    # Load it into a dataframe, specifying a delimiter and column names
    dfs.append(pd.read_csv(data_file['Body'], 
                           delimiter = " ", 
                           names=["record_id", "timestamp", "vin", "lon", "lat", "speed"]))

# Concatenate the resulting dataframes.
data = pd.concat(dfs)
print(data.groupby(['vin'])['speed'].max())
